#  Step 1: Dataset Preparation


In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

# Load the Amazon Product Reviews dataset

In [34]:
data = pd.read_csv('ratings_Beauty.csv')

In [35]:
encoder = LabelEncoder()
data['UserId'] = encoder.fit_transform(data['UserId'])
data['ProductId'] = encoder.fit_transform(data['ProductId'])

# Split the data into training and testing sets

In [36]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


# Create the kNN model

In [37]:
# Create the kNN model
k = 5  # Number of neighbors to consider
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k)

# Fit the model to the training data
knn_model.fit(train_data[['UserId', 'ProductId']])


NearestNeighbors(algorithm='brute', metric='cosine')

# Define a function to recommend products to a user

In [38]:
# Define a function to recommend products to a user
def recommend_products(UserId, num_recommendations):
    # Find the k nearest neighbors of the user
    _, indices = knn_model.kneighbors(train_data[train_data['UserId'] ==UserId][['UserId', 'ProductId']])

    # Get the product IDs of the nearest neighbors
    neighbor_ids = train_data.iloc[indices[0]]['ProductId'].values

    # Count the occurrence of each product and sort by count
    product_counts = pd.DataFrame(neighbor_ids, columns=['ProductId']).groupby('ProductId').size().reset_index(name='count')
    product_counts = product_counts.sort_values(by='count', ascending=False)

    # Get the top recommendations
    recommendations = product_counts.head(num_recommendations)

    # Convert product IDs back to original values
    recommendations['ProductId'] = encoder.inverse_transform(recommendations['ProductId'])

    return recommendations


# Specify the user ID for whom you want to generate recommendations

In [39]:
UserId = 100 # Specify the user ID for whom you want to generate recommendations
num_recommendations = 10 # Number of recommendations to generate

recommended_products = recommend_products(UserId, num_recommendations)
print(recommended_products)


    ProductId  count
0  B003BMGA6C      1
1  B00508Y0EO      1
2  B005Y5VL4U      1
3  B008G8MRLS      1
4  B009YA758M      1
